In [1]:
import requests
from scrapy.http import TextResponse
import re
import json

In [2]:
mainUrl = "https://www.ah.be/allerhande/recepten-zoeken?format=json&No="
user_agent = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/58: .0.3029.110 Chrome/58.0.3029.110 Safari/537.36'}
page = 1
recipePrefix = "https://www.ah.be/allerhande/recept"


In [3]:
def getResponse(url, user_agent):
    r = requests.get(url, headers=user_agent)
    return TextResponse(r.url, body=r.text, encoding='utf-8')

In [4]:
def getReceptUrls(previousIndex):
    resp = getResponse(mainUrl + str(previousIndex), user_agent)
    jsonObject = json.loads(resp.text)
    urls = []
    for record in jsonObject["contents"][0]["contentArea"][2]["mainContent"][1]["contents"][0]["records"]:
        urls.append(recipePrefix + record["detailsAction"]["recordState"])
    return urls

In [5]:
def title(response):
    title = response.xpath('//h1 [@itemprop="name"]/text()').extract()[0]
    #Remove soft hyphens
    title = title.replace('­', '')
    return title

In [6]:
def description(response):
    description = response.xpath('//h1 [@class="hidden-phones"]/text()').extract()[0]

In [7]:
def rating(response):
    rating = 0
    ratings = response.xpath('//div [@class="rating "]')[0]
    for star in ratings.xpath('.//ul/li/div/@class').extract():
        if bool(re.search('active', star)):
            rating = rating + 1
    return rating

In [14]:
def votes(response):
    votes = response.xpath('//div [@class="rating "]/span [@class="all-rates"]/span/text()').extract()
    if len(votes) > 0:
        return int(votes[0].replace('.',''))
    else:
        return -1

In [9]:
# Inhoud ingredienten
def ingredients(response):
    ingredients = []
    for el in response.xpath('//li [@itemprop="ingredients"]'):
        ingredient = {}
        component = el.xpath('.//a/@data-search-term').extract()
        amount = el.xpath('.//a/@data-quantity').extract()
        unit = el.xpath('.//a/@data-quantity-unit-singular').extract()
        description = el.xpath('.//a/@data-default-label').extract()
        if len(description) > 0:
            ingredient['description']= description[0]
        if len(component) > 0:
            ingredient['component']= component[0]
            if len(amount) > 0:
                ingredient['amount']= amount[0]
            if len(unit) > 0:
                ingredient['unit']= unit[0]
            ingredients.append(ingredient) 
    return ingredients

In [10]:
def instructions(response):
    instructions = []
    for step in response.xpath('//section [@class="preparation"]/ol/li/text()').extract():
        instructions.append(step)
    return instructions

In [11]:
def labels(response):
    labels = []
    for label in response.xpath('//section [@class="tags"]/ul [@class="tags"]/li/a/text()').extract():
        labels.append(label)
    return labels

In [12]:
def createRecipe(url):
    response = getResponse(url, user_agent)
    recipe = {
            'url':url,
            'name':title(response),
            'rating':rating(response),
            'maxRating':5,
            'votes':votes(response),
            'ingredients':ingredients(response),
            'instructions':instructions(response),
            'labels':labels(response)
        }
    return recipe

In [16]:
data = {}
data['recipes']=[]
index = 0
while True:
    print("PAGE " + str(page))
    receptPages = getReceptUrls(index)
    if (len(receptPages) == 0):
        break
    #alle recepten op een pagina
    for receptUrl in receptPages:
        index = index + 1
        print(receptUrl)
        print("RECEPT " + str(index))
        data['recipes'].append(createRecipe(receptUrl))
    
    if page%100 == 0:
        print('EXPORTING TO ' + 'data' + str(page) + '.txt')
        with open('data' + str(page) + '.txt', 'w', encoding='utf8') as outfile:  
            json.dump(data, outfile, ensure_ascii=False)
        data['recipes']=[]    
    
    page = page+ 1

print('EXPORTING TO ' + 'data' + str(page) + '.txt')
with open('data' + str(page) + '.txt', 'w', encoding='utf8') as outfile:  
    json.dump(data, outfile, ensure_ascii=False)
print("DONE")

PAGE 301
https://www.ah.be/allerhande/recept/R-R1190455/traktatie-van-eetbare-slakken
RECEPT 1
https://www.ah.be/allerhande/recept/R-R1190454/traktatie-van-appels-in-monstervorm
RECEPT 2
https://www.ah.be/allerhande/recept/R-R1190453/galaxy-slush-cocktail
RECEPT 3
https://www.ah.be/allerhande/recept/R-R1190452/vegan-nachos-uit-de-oven-met-tomatensalsa
RECEPT 4
https://www.ah.be/allerhande/recept/R-R1190451/vegan-banh-mi
RECEPT 5
https://www.ah.be/allerhande/recept/R-R1190450/taco-s-met-crispy-polenta-halloumi
RECEPT 6
https://www.ah.be/allerhande/recept/R-R1190449/noten-rozijnenbrood
RECEPT 7
https://www.ah.be/allerhande/recept/R-R1190448/pizza-tartufato
RECEPT 8
https://www.ah.be/allerhande/recept/R-R1190447/pastasalade-met-tonijn
RECEPT 9
https://www.ah.be/allerhande/recept/R-R1190446/naan-met-zalm-in-sojasaus-en-sojabonenspread
RECEPT 10
https://www.ah.be/allerhande/recept/R-R1190445/kip-en-groenten-van-de-bakplaat
RECEPT 11
https://www.ah.be/allerhande/recept/R-R1190444/sushisalade

https://www.ah.be/allerhande/recept/R-R1190303/aspergetagliatelle-met-warmgerookte-zalm-en-ei
RECEPT 92
https://www.ah.be/allerhande/recept/R-R1190302/groenteplaattaart-met-rauwe-ham
RECEPT 93
https://www.ah.be/allerhande/recept/R-R1190301/aspergepizza
RECEPT 94
https://www.ah.be/allerhande/recept/R-R1190300/geroosterde-tomatensoep
RECEPT 95
https://www.ah.be/allerhande/recept/R-R1190299/lentemaaltijdsalade-van-quinoa-en-biet
RECEPT 96
PAGE 305
https://www.ah.be/allerhande/recept/R-R1190298/lentepasta-met-gerookte-kip
RECEPT 97
https://www.ah.be/allerhande/recept/R-R1190297/volkorenbrood-met-boerenomelet
RECEPT 98
https://www.ah.be/allerhande/recept/R-R1190296/vegetarische-lenteminestrone
RECEPT 99
https://www.ah.be/allerhande/recept/R-R1190295/zilvervliesrijst-met-gegrilde-kiphaasjes
RECEPT 100
https://www.ah.be/allerhande/recept/R-R1190294/volkorenpasta-met-groene-groenten-en-cottage-cheese
RECEPT 101
https://www.ah.be/allerhande/recept/R-R1190293/linzenschotel-met-cajunkruiden
RECEP

https://www.ah.be/allerhande/recept/R-R1190166/het-perfecte-roerei
RECEPT 179
https://www.ah.be/allerhande/recept/R-R1190165/homemade-kipnuggets
RECEPT 180
https://www.ah.be/allerhande/recept/R-R1190164/salade-van-geplukte-kip-met-walnoten-en-druiven
RECEPT 181
https://www.ah.be/allerhande/recept/R-R1190163/brunchbol-met-kipfilet-blauwaderkaas-jam-en-meslcun
RECEPT 182
https://www.ah.be/allerhande/recept/R-R1190162/prei-slastamppot-met-zoete-aardappel-en-verse-worst
RECEPT 183
https://www.ah.be/allerhande/recept/R-R1190161/groentepizza-met-kipshoarma-en-pittige-wortelsalade
RECEPT 184
https://www.ah.be/allerhande/recept/R-R1190142/krokante-kip-met-aardappeltjes-en-koolrabisla
RECEPT 185
https://www.ah.be/allerhande/recept/R-R1190141/groenteburger-met-rozemarijnaardappeltjes
RECEPT 186
https://www.ah.be/allerhande/recept/R-R1190140/7-granen-wortelkoekjes
RECEPT 187
https://www.ah.be/allerhande/recept/R-R1190121/penne-met-groentesaus-chorizo-en-garnalen
RECEPT 188
https://www.ah.be/aller

https://www.ah.be/allerhande/recept/R-R1189903/courgettespaghetti-met-tomaat-en-pistache
RECEPT 264
PAGE 312
https://www.ah.be/allerhande/recept/R-R1189902/kip-groentetajine-met-couscous-en-munt
RECEPT 265
https://www.ah.be/allerhande/recept/R-R1189901/teriyaki-noedels-met-courgette-en-gehakt
RECEPT 266
https://www.ah.be/allerhande/recept/R-R1189900/vegan-creme-brulee
RECEPT 267
https://www.ah.be/allerhande/recept/R-R1189899/sushi-met-fruit
RECEPT 268
https://www.ah.be/allerhande/recept/R-R1189898/lavacakejes-met-pindakaas-en-chocolade
RECEPT 269
https://www.ah.be/allerhande/recept/R-R1189897/kurkumapannenkoek-met-appel
RECEPT 270
https://www.ah.be/allerhande/recept/R-R1189896/kurkumadressing
RECEPT 271
https://www.ah.be/allerhande/recept/R-R1189895/gender-reveal-taart
RECEPT 272
https://www.ah.be/allerhande/recept/R-R1189894/zalmpakketje-met-zeewier-en-volkoren-noedels
RECEPT 273
https://www.ah.be/allerhande/recept/R-R1189893/zeewiertagliatelle-met-tomaten-groentesaus-en-kip
RECEPT 27

https://www.ah.be/allerhande/recept/R-R1189774/eendenravioli-met-zachte-eidooier-en-truffelsaus
RECEPT 354
https://www.ah.be/allerhande/recept/R-R1189773/vegan-meatloaf
RECEPT 355
https://www.ah.be/allerhande/recept/R-R1189772/duo-vijgenrol-met-amandelen-en-abrikozen-of-chocola
RECEPT 356
https://www.ah.be/allerhande/recept/R-R1189771/vegan-tiramisu
RECEPT 357
https://www.ah.be/allerhande/recept/R-R1189770/vegan-roomkaas-voor-de-borrel
RECEPT 358
https://www.ah.be/allerhande/recept/R-R1189769/fruitsterren-op-een-stokje
RECEPT 359
https://www.ah.be/allerhande/recept/R-R1189768/vegan-meringue-kerstkrans
RECEPT 360
PAGE 316
https://www.ah.be/allerhande/recept/R-R1189767/brownie-aardbei-kerstmannetje-op-een-stokje
RECEPT 361
https://www.ah.be/allerhande/recept/R-R1189766/snowglobe-met-koekhuisje
RECEPT 362
https://www.ah.be/allerhande/recept/R-R1189765/kleurenwraps-op-een-stokje
RECEPT 363
https://www.ah.be/allerhande/recept/R-R1189764/ricotta-gnocchi
RECEPT 364
https://www.ah.be/allerhand

...

https://www.ah.be/allerhande/recept/R-R8358/koude-dikke-tomatencreme-uit-cordoba
RECEPT 16542
https://www.ah.be/allerhande/recept/R-R8356/gemengde-gefrituurde-vis-met-knoflookdressing
RECEPT 16543
https://www.ah.be/allerhande/recept/R-R8354/little-gemsalade-met-tomaten-dressing
RECEPT 16544
https://www.ah.be/allerhande/recept/R-R8352/vers-en-gedroogd-fruit-in-rode-wijnsiroop
RECEPT 16545
https://www.ah.be/allerhande/recept/R-R8350/witte-rijst-met-sherry-en-pijnboompitten
RECEPT 16546
https://www.ah.be/allerhande/recept/R-R8348/raapstelensalade-met-avocado-en-peterseliedressing
RECEPT 16547
https://www.ah.be/allerhande/recept/R-R8346/lasagnepakketjes-met-raapstelen
RECEPT 16548
https://www.ah.be/allerhande/recept/R-R8344/raapsteelschotel-uit-de-wok
RECEPT 16549
https://www.ah.be/allerhande/recept/R-R8342/rodekool-met-appelstroop-en-kaneel
RECEPT 16550
https://www.ah.be/allerhande/recept/R-R8340/couscouspilav-met-kipfilet-en-rode-wijnjus
RECEPT 16551
https://www.ah.be/allerhande/recept/R

https://www.ah.be/allerhande/recept/R-R8110/wildbouillon-met-gerookte-eendenborst
RECEPT 16630
https://www.ah.be/allerhande/recept/R-R8108/salade-van-serranoham-en-granaatappel
RECEPT 16631
https://www.ah.be/allerhande/recept/R-R8106/groene-tagliatelle-met-blauwe-kaassaus-en-croutons
RECEPT 16632
PAGE 994
https://www.ah.be/allerhande/recept/R-R8104/gevulde-paprika-met-geitenkaas-en-ansjovis
RECEPT 16633
https://www.ah.be/allerhande/recept/R-R8102/rode-paprikasoep-met-mascarpone-en-pizzasterren
RECEPT 16634
https://www.ah.be/allerhande/recept/R-R8100/gravad-lachs-op-erwtjessalade-met-kaviaar
RECEPT 16635
https://www.ah.be/allerhande/recept/R-R8098/oesters-op-een-bedje-van-spinazie-met-zongedroogde-tomaat
RECEPT 16636
https://www.ah.be/allerhande/recept/R-R8096/antipasti
RECEPT 16637
https://www.ah.be/allerhande/recept/R-R8094/spaanse-boerenkool
RECEPT 16638
https://www.ah.be/allerhande/recept/R-R8092/teriyaki-kalkoen-met-champignons-en-waterkers
RECEPT 16639
https://www.ah.be/allerhande

https://www.ah.be/allerhande/recept/R-R7896/tortiglioni-met-ham-kaas
RECEPT 16722
https://www.ah.be/allerhande/recept/R-R7892/tabbouleh-met-tomaatjes-en-kaas
RECEPT 16723
https://www.ah.be/allerhande/recept/R-R7890/boerenomelet
RECEPT 16724
https://www.ah.be/allerhande/recept/R-R7888/witte-bonensalade-met-tonijn
RECEPT 16725
https://www.ah.be/allerhande/recept/R-R7886/frittata
RECEPT 16726
https://www.ah.be/allerhande/recept/R-R7884/kruidige-gehaktballetjes
RECEPT 16727
https://www.ah.be/allerhande/recept/R-R7882/auberginerolletjes-met-pijnboompitten
RECEPT 16728
PAGE 998
https://www.ah.be/allerhande/recept/R-R7880/basboesa
RECEPT 16729
https://www.ah.be/allerhande/recept/R-R7878/tuinbonensalade
RECEPT 16730
https://www.ah.be/allerhande/recept/R-R7876/worteldip-met-pitatriangels
RECEPT 16731
https://www.ah.be/allerhande/recept/R-R7874/pittige-gebakken-garnalen
RECEPT 16732
https://www.ah.be/allerhande/recept/R-R7872/marokkaanse-kippenpastei-bisteeya
RECEPT 16733
https://www.ah.be/aller

https://www.ah.be/allerhande/recept/R-R7698/couscous-met-shoarma-en-sperziebonen
RECEPT 16815
https://www.ah.be/allerhande/recept/R-R7696/jacket-potato-met-tonijn-en-spinaziesalade
RECEPT 16816
https://www.ah.be/allerhande/recept/R-R7694/rocketsalade-met-citroenkip
RECEPT 16817
https://www.ah.be/allerhande/recept/R-R7692/tomatensoep
RECEPT 16818
https://www.ah.be/allerhande/recept/R-R7690/lauwwarme-aardappelmaaltijdsalade
RECEPT 16819
https://www.ah.be/allerhande/recept/R-R7688/kabeljauw-garnalenschotel-uit-de-oven
RECEPT 16820
https://www.ah.be/allerhande/recept/R-R7686/pittige-pitabroodjes
RECEPT 16821
https://www.ah.be/allerhande/recept/R-R7684/frisse-vissandwiches
RECEPT 16822
https://www.ah.be/allerhande/recept/R-R7682/koude-komkommersoep
RECEPT 16823
https://www.ah.be/allerhande/recept/R-R7680/aardbeiensalade-met-geitenkaas
RECEPT 16824
PAGE 1002
https://www.ah.be/allerhande/recept/R-R7678/roeibroodjes
RECEPT 16825
https://www.ah.be/allerhande/recept/R-R7676/pannenkoekrolletjes
R

https://www.ah.be/allerhande/recept/R-R7500/gegratineerde-witlofblaadjes
RECEPT 16911
https://www.ah.be/allerhande/recept/R-R7498/witlofbeignets
RECEPT 16912
https://www.ah.be/allerhande/recept/R-R7496/roergebakken-witlof
RECEPT 16913
https://www.ah.be/allerhande/recept/R-R7494/romige-witlof
RECEPT 16914
https://www.ah.be/allerhande/recept/R-R7492/bolognesesaus
RECEPT 16915
https://www.ah.be/allerhande/recept/R-R7490/courgettebootjes
RECEPT 16916
https://www.ah.be/allerhande/recept/R-R7488/kersen-ricottamousse
RECEPT 16917
https://www.ah.be/allerhande/recept/R-R7484/macaroni-met-boemboe-ham-en-kaas
RECEPT 16918
https://www.ah.be/allerhande/recept/R-R7482/franse-zuurkoolschotel
RECEPT 16919
https://www.ah.be/allerhande/recept/R-R7480/thaise-vispakketjes
RECEPT 16920
PAGE 1006
https://www.ah.be/allerhande/recept/R-R7478/korianderpasta-met-mosselen
RECEPT 16921
https://www.ah.be/allerhande/recept/R-R7476/pitabroodjes-met-lamsvlees-en-tomatenrelish
RECEPT 16922
https://www.ah.be/allerhande